In [1]:
from nltk.tokenize import word_tokenize
import torch
import torch.nn as nn
import torch.nn.functional as F
import math

In [2]:
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

In [3]:
with open('../data/input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [4]:
words = sorted(list(set(word_tokenize(text))))
vocab_size = len(words)

# create a mapping from characters to integers
stoi = { w:i for i,w in enumerate(words) }
itos = { i:w for i,w in enumerate(words) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ' '.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

# encode the entire text dataset and store it into a torch.Tensor
data = torch.tensor(encode(word_tokenize(text)), dtype=torch.long)

In [5]:
# Let's now split up the data into train and validation sets
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

In [6]:
torch.manual_seed(1337)
batch_size = 4 # How many independent sequences will be process in parallel?
block_size = 8 # What is the maximum context length for predictions?

In [7]:
def get_batch(batch_size, block_size, split):
    data = train_data if split == 'train' else val_data
    
    sindex = 0
    eindex = batch_size * block_size
    while eindex < (len(data) - block_size):
        # Extract batch_size * block_size tokens in the data
        xb = data[sindex: eindex]
        yb = data[sindex+1: eindex+1]
        # Reshape the batch by the shape (batch_size, block_size)
        xb = xb.reshape(batch_size, block_size)
        yb = yb.reshape(batch_size, block_size)
        # Update the indexes for extracting tokens sections
        temp = eindex
        eindex = eindex + block_size * batch_size
        sindex = temp
        yield xb.to(DEVICE), yb.to(DEVICE)
    
    # For the last batch that cannot be batched as in the size (batch_size, block_size)
    # Maintain the 'block_size' dimension
    if eindex >= (len(data) - block_size):
        # The number of useable completed sample including 'block_size' elements
        num_sample = (len(data) - sindex) // block_size 
        if num_sample >= 1:
            xb = data[sindex: sindex + (num_sample * block_size)]
            yb = data[sindex+1: sindex + (num_sample * block_size)+1]
            xb = xb.reshape(num_sample, block_size)
            yb = yb.reshape(num_sample, block_size)
            yield xb.to(DEVICE), yb.to(DEVICE)

In [8]:
for i, batch in enumerate(get_batch(batch_size, block_size, 'train')):
    xb, yb = batch
    if i < 1:
        print('inputs:')
        print(xb.shape)
        print(xb)
        print('targets:')
        print(yb.shape)
        print(yb)

inputs:
torch.Size([4, 8])
tensor([[ 1152,   709,   223,   482, 13877, 10480,  3440,  7080],
        [  219,  7604,  8993, 12087,   221,   323,   223,  2520],
        [  219, 12087,   221,  1152,   709,   223,  3053,  3512],
        [ 3324, 11053, 10791, 13010,  5723, 12819, 13010,  6533]],
       device='cuda:0')
targets:
torch.Size([4, 8])
tensor([[  709,   223,   482, 13877, 10480,  3440,  7080,   219],
        [ 7604,  8993, 12087,   221,   323,   223,  2520,   219],
        [12087,   221,  1152,   709,   223,  3053,  3512,  3324],
        [11053, 10791, 13010,  5723, 12819, 13010,  6533,   225]],
       device='cuda:0')


In [9]:
# for b in range(batch_size): # batch dimension
#     print(f'batch {b+1}/{batch_size}')
#     for t in range(block_size): # time dimension
#         context = xb[b, :t+1]
#         target = yb[b,t]
#         print(f"when input is {context.tolist()} the target: {target}")
#     print()

In [10]:
# hyperparameters
batch_size = 32 # how many independent sequences will we process in parallel?
block_size = 128 # what is the maximum context length for predictions?
max_iters = 2000
eval_interval = 500
learning_rate = 3e-4
eval_iters = 150
n_embd = 256
n_head = 8
n_layer = 4
dropout = 0.3
# ------------

In [11]:
class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        # input of size (batch, time-step, channels)
        # output of size (batch, time-step, head size)
        B,T,C = x.shape
        k = self.key(x)   # (B,T,hs)
        q = self.query(x) # (B,T,hs)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * k.shape[-1]**-0.5 # (B, T, hs) @ (B, hs, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,hs)
        out = wei @ v # (B, T, T) @ (B, T, hs) -> (B, T, hs)
        return out

class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(head_size * num_heads, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

class GPTLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)

        # better init, not covered in the original GPT video, but important, will cover in followup video
        self.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=DEVICE)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

In [12]:
model = GPTLanguageModel()
model.to(DEVICE)

GPTLanguageModel(
  (token_embedding_table): Embedding(14310, 256)
  (position_embedding_table): Embedding(128, 256)
  (blocks): Sequential(
    (0): Block(
      (sa): MultiHeadAttention(
        (heads): ModuleList(
          (0-7): 8 x Head(
            (key): Linear(in_features=256, out_features=32, bias=False)
            (query): Linear(in_features=256, out_features=32, bias=False)
            (value): Linear(in_features=256, out_features=32, bias=False)
            (dropout): Dropout(p=0.3, inplace=False)
          )
        )
        (proj): Linear(in_features=256, out_features=256, bias=True)
        (dropout): Dropout(p=0.3, inplace=False)
      )
      (ffwd): FeedFoward(
        (net): Sequential(
          (0): Linear(in_features=256, out_features=1024, bias=True)
          (1): ReLU()
          (2): Linear(in_features=1024, out_features=256, bias=True)
          (3): Dropout(p=0.3, inplace=False)
        )
      )
      (ln1): LayerNorm((256,), eps=1e-05, elementwise_affi

In [13]:
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3)

In [14]:
batch_size = 16
block_size = 32
epochs = 5
num_checkpoints = 4
train_eval_iters = math.ceil(len(train_data) / (batch_size * block_size))
val_eval_iters = math.ceil(len(val_data) / (batch_size * block_size))
print(train_eval_iters, val_eval_iters)

checkpoints = [i for i in range(0, train_eval_iters, train_eval_iters // num_checkpoints)]
print(checkpoints)

448 50
[0, 112, 224, 336]


In [15]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        eval_iters = train_eval_iters if split == 'train' else val_eval_iters
        losses = torch.zeros(eval_iters)
        for i, batch in enumerate(get_batch(batch_size, block_size, split)):
            X, Y = batch
            _, loss = model(X, Y)
            losses[i] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [18]:
from tqdm import tqdm
from colorama import Fore, Style

for e in range(epochs):
    print(f'Epoch: {e+1}/{epochs}:=====================')
    
    for i, batch in enumerate(tqdm(get_batch(batch_size, block_size, 'train'), total=train_eval_iters)):
        
        if i in checkpoints or i == train_eval_iters - 1:
            losses = estimate_loss()
            print(Fore.GREEN + 
                  f"step {i+1}/{train_eval_iters}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")
            
        xb, yb = batch
        logits, loss = model(xb, yb)
            
        optimizer.zero_grad(set_to_none=True)
        loss.backward()
        optimizer.step()
    
    print(Style.RESET_ALL)

Epoch: 1/5:=====================


  1%|          | 5/448 [00:04<05:31,  1.34it/s]

step 1/448: train loss 5.4289, val loss 5.9748


 26%|██▌       | 117/448 [00:12<01:28,  3.75it/s]

step 113/448: train loss 5.2970, val loss 6.2502


...

step 225/448: train loss 5.0295, val loss 6.3916


 76%|███████▋  | 342/448 [00:28<00:29,  3.63it/s]

step 337/448: train loss 5.0773, val loss 6.3331


100%|██████████| 448/448 [00:36<00:00, 12.18it/s]

step 448/448: train loss 5.0797, val loss 6.2089



In [19]:
start_token_string = 'Well'
start_token = torch.tensor(stoi[start_token_string], dtype=torch.long, device=DEVICE).view(-1, 1)

print(decode(model.generate(start_token, max_new_tokens=100)[0].tolist()))

Well , sir . But , to have an prisoner sooner cormorant Citizen : vex 'd by the breasts Whom any Baptista 's daughter waked , a play And I see . GRUMIO : There you serve back ; And this report to hand ; with me well lengthens love for he ? ABHORSON : or either , Signior il Hortensio , sir , man . For madly Neapolitan , Honour for what 's love you , peace ? BAPTISTA : Methinks an beauteous duties , And in me with me Before you bid good ben languages for thy stitchery .


In [21]:
params = sum(p.numel() for p in model.parameters())
print(params/1e6, 'M parameters')

10.530278 M parameters


In [25]:
from datetime import date

model_size_str = str(int(params/1e6))
model_file_name = 'word_transformer' + model_size_str + 'M-'+ str(date.today()) +'.pth'
model_save_path = '../params/transformer/' + model_file_name
torch.save(model.state_dict(), model_save_path)
print("The model saved successfully.")

The model saved successfully.
